In [2]:
from features_and_svm_functions import *
import warnings
import yaml # for the config and output files
from yaml.loader import SafeLoader


2022-12-22 11:33:50.959869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 11:33:51.343803: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-22 11:33:51.352373: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-22 11:33:51.352406: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
# Path to input files : they contain the paramters we tuned 
# Modify it with the actual input files you want to use 
path_to_input_all    = '../../Data/Inputs/After_Tuning/Input_baseline.yaml' #'Inputs/After_Tuning/Input_tuned_all.yaml'
path_to_input_native = '../../Data/Inputs/After_Tuning/Input_baseline.yaml'#'Inputs/After_Tuning/Input_tuned_native.yaml'
path_to_input_nonnat = '../../Data/Inputs/After_Tuning/Input_baseline.yaml'#'Inputs/After_Tuning/Input_tuned_nonnative.yaml'
path_to_input_max   = '../../Data/Inputs/After_Tuning/Input_max.yaml'

# Generate train and test data for test phase #

In [3]:
## Read config file :
with open(path_to_input_max, 'r') as f:
    config_max = list(yaml.load_all(f, Loader=SafeLoader))
config_max = config_max[0]

In [4]:
seed = config_max['seed']

In [5]:
## Load data of the testing set
test_100_native_df = pd.read_parquet('../../Data/Test/100native_english',engine='fastparquet')
test_100_nonnat_df = pd.read_parquet('../../Data/Test/100non_native_english',engine='fastparquet')
test_100_native_df['proficiency'] = "N" # N = native
test_100_nonnat_df['proficiency'] = "L" # L = learner

In [6]:
# Select 3*(30 natives, 30 non-natives)
num_native_authors_to_sample = 30
num_nonnat_authors_to_sample = 30

test_100_nonnat_df = test_100_nonnat_df.sample(frac=1)
test_100_native_df = test_100_native_df.sample(frac=1)

test_native_s1 = test_100_native_df.iloc[0:30,:]   # trial 1
test_native_s2 = test_100_native_df.iloc[30:60,:]  # trial 2 
test_native_s3 = test_100_native_df.iloc[60:90,:]  # trial 3

test_nonnat_s1 = test_100_nonnat_df.iloc[0:30,:]  # trial 1
test_nonnat_s2 = test_100_nonnat_df.iloc[30:60,:] # trial 2
test_nonnat_s3 = test_100_nonnat_df.iloc[60:90,:] # trial 3


In [7]:
# build the three sets of (30 native + 30 nonnat) to train and 3*(3*(30+30)) corresponding cohorts 
s1_cohort_all, s1_cohort_native, s1_cohort_nonnat = build_cohorts(30,30,test_native_s1,test_nonnat_s1,seed)
s2_cohort_all, s2_cohort_native, s2_cohort_nonnat = build_cohorts(30,30,test_native_s2,test_nonnat_s2,seed)
s3_cohort_all, s3_cohort_native, s3_cohort_nonnat = build_cohorts(30,30,test_native_s3,test_nonnat_s3,seed)

Tokenizing feeds...


100%|██████████| 1200/1200 [02:12<00:00,  9.06it/s]


Feeds tokenized in 132.41242003440857 seconds
Tokenizing feeds...


100%|██████████| 1200/1200 [02:16<00:00,  8.76it/s]


Feeds tokenized in 136.97628664970398 seconds
Tokenizing feeds...


100%|██████████| 1200/1200 [02:17<00:00,  8.70it/s]

Feeds tokenized in 137.9188461303711 seconds


In [10]:
s1_cohort_all.to_pickle('dataset/Test/s1_cohort_all',protocol=4)
s1_cohort_native.to_pickle('dataset/Test/s1_cohort_native',protocol=4) 
s1_cohort_nonnat.to_pickle('dataset/Test/s1_cohort_nonnat',protocol=4)

s2_cohort_all.to_pickle('dataset/Test/s2_cohort_all',protocol=4)
s2_cohort_native.to_pickle('dataset/Test/s2_cohort_native',protocol=4)
s2_cohort_nonnat.to_pickle('dataset/Test/s2_cohort_nonnat',protocol=4)

s3_cohort_all.to_pickle('dataset/Test/s3_cohort_all',protocol=4)
s3_cohort_native.to_pickle('dataset/Test/s3_cohort_native',protocol=4) 
s3_cohort_nonnat.to_pickle('dataset/Test/s3_cohort_nonnat',protocol=4)

In [ ]:
# extract features for these 3 datasets :
##### takes about 3-4 hours if done so :
#X_train_alls1,X_test_alls1,y_train_alls1,y_test_alls1 = extract_features(s1_cohort_all,config_max,"all_subset1")
#X_train_alls2,X_test_alls2,y_train_alls2,y_test_alls2 = extract_features(s2_cohort_all,config_max,"subset2")
#X_train_alls3,X_test_alls3,y_train_alls3,y_test_alls3 = extract_features(s3_cohort_all,config_max,"subset3")

#X_train_natives1,X_test_natives1,y_train_natives1,y_test_alls1 = extract_features(s1_cohort_all,config_max,"all_subset1")
#X_train_natives2,X_test_natives2,y_train_natives2,y_test_alls2 = extract_features(s2_cohort_all,config_max,"subset2")
#X_train_natives3,X_test_natives3,y_train_natives3,y_test_alls3 = extract_features(s3_cohort_all,config_max,"subset3")

#X_train_alls1,X_test_alls1,y_train_alls1,y_test_alls1 = extract_features(s1_cohort_all,config_max,"all_subset1")
#X_train_alls2,X_test_alls2,y_train_alls2,y_test_alls2 = extract_features(s2_cohort_all,config_max,"subset2")
#X_train_alls3,X_test_alls3,y_train_alls3,y_test_alls3 = extract_features(s3_cohort_all,config_max,"subset3")

##### Otherwise run feature_extraction_for_evaluation_stage.py with arguments 1, 2 and 3 in parallel (2 hours)
##### It will save the dataframes with the rigth names etc

# EVALUATION STAGE #

In [72]:
# Read specific config files for specific models

# Config for best mixed classification
with open(path_to_input_all, 'r') as f:
    config_all = list(yaml.load_all(f, Loader=SafeLoader))
config_all = config_all[0]

# Config for best native classification
with open(path_to_input_native, 'r') as f:
    config_native = list(yaml.load_all(f, Loader=SafeLoader))
config_native = config_native[0]

# Config for best nonnative classification
with open(path_to_input_nonnat, 'r') as f:
    config_nonnat = list(yaml.load_all(f, Loader=SafeLoader))
config_nonnat = config_nonnat[0]

In [4]:
# Load the train and test data
# Divided in three samples --> we will average the results 
# Each model we will train will be trained and tested for the same authors = need just 3 sets of t
path_to_dataset = '../../Data/Test/'

# COHORT ALL, data for trials 1 2 and 3
X_train_all_s1 = pd.read_parquet(path_to_dataset+'X_train_all_s1',engine='fastparquet')
X_test_all_s1  = pd.read_parquet(path_to_dataset+'X_test_all_s1',engine='fastparquet')
y_train_all_s1 = pd.read_parquet(path_to_dataset+'y_train_all_s1',engine='fastparquet')
y_test_all_s1  = pd.read_parquet(path_to_dataset+'y_test_all_s1',engine='fastparquet')

X_train_all_s2 = pd.read_parquet(path_to_dataset+'X_train_all_s2',engine='fastparquet')
X_test_all_s2  = pd.read_parquet(path_to_dataset+'X_test_all_s2',engine='fastparquet')
y_train_all_s2 = pd.read_parquet(path_to_dataset+'y_train_all_s2',engine='fastparquet')
y_test_all_s2  = pd.read_parquet(path_to_dataset+'y_test_all_s2',engine='fastparquet')

X_train_all_s3 = pd.read_parquet(path_to_dataset+'X_train_all_s3',engine='fastparquet')
X_test_all_s3  = pd.read_parquet(path_to_dataset+'X_test_all_s3',engine='fastparquet')
y_train_all_s3 = pd.read_parquet(path_to_dataset+'y_train_all_s3',engine='fastparquet')
y_test_all_s3  = pd.read_parquet(path_to_dataset+'y_test_all_s3',engine='fastparquet')



# COHORT NATIVE,, data for trials 1 2 and 3 
X_train_native_s1 = pd.read_parquet(path_to_dataset+'X_train_native_s1',engine='fastparquet')
X_test_native_s1  = pd.read_parquet(path_to_dataset+'X_test_native_s1',engine='fastparquet')
y_train_native_s1 = pd.read_parquet(path_to_dataset+'y_train_native_s1',engine='fastparquet')
y_test_native_s1  = pd.read_parquet(path_to_dataset+'y_test_native_s1',engine='fastparquet')

X_train_native_s2 = pd.read_parquet(path_to_dataset+'X_train_native_s2',engine='fastparquet')
X_test_native_s2  = pd.read_parquet(path_to_dataset+'X_test_native_s2',engine='fastparquet')
y_train_native_s2 = pd.read_parquet(path_to_dataset+'y_train_native_s2',engine='fastparquet')
y_test_native_s2  = pd.read_parquet(path_to_dataset+'y_test_native_s2',engine='fastparquet')

X_train_native_s3 = pd.read_parquet(path_to_dataset+'X_train_native_s3',engine='fastparquet')
X_test_native_s3  = pd.read_parquet(path_to_dataset+'X_test_native_s3',engine='fastparquet')
y_train_native_s3 = pd.read_parquet(path_to_dataset+'y_train_native_s3',engine='fastparquet')
y_test_native_s3  = pd.read_parquet(path_to_dataset+'y_test_native_s3',engine='fastparquet')



# COHORT NONNAT, , data for trials 1 2 and 3
X_train_nonnat_s1 = pd.read_parquet(path_to_dataset+'X_train_nonnat_s1',engine='fastparquet')
X_test_nonnat_s1  = pd.read_parquet(path_to_dataset+'X_test_nonnat_s1',engine='fastparquet')
y_train_nonnat_s1 = pd.read_parquet(path_to_dataset+'y_train_nonnat_s1',engine='fastparquet')
y_test_nonnat_s1  = pd.read_parquet(path_to_dataset+'y_test_nonnat_s1',engine='fastparquet')

X_train_nonnat_s2 = pd.read_parquet(path_to_dataset+'X_train_nonnat_s2',engine='fastparquet')
X_test_nonnat_s2  = pd.read_parquet(path_to_dataset+'X_test_nonnat_s2',engine='fastparquet')
y_train_nonnat_s2 = pd.read_parquet(path_to_dataset+'y_train_nonnat_s2',engine='fastparquet')
y_test_nonnat_s2  = pd.read_parquet(path_to_dataset+'y_test_nonnat_s2',engine='fastparquet')

X_train_nonnat_s3 = pd.read_parquet(path_to_dataset+'X_train_nonnat_s3',engine='fastparquet')
X_test_nonnat_s3  = pd.read_parquet(path_to_dataset+'X_test_nonnat_s3',engine='fastparquet')
y_train_nonnat_s3 = pd.read_parquet(path_to_dataset+'y_train_nonnat_s3',engine='fastparquet')
y_test_nonnat_s3  = pd.read_parquet(path_to_dataset+'y_test_nonnat_s3',engine='fastparquet')


In [73]:
# TRIAL 1

# Performing the test for configuration 'all' :

# ...on all
res_all_on_all_1    = classify_with_input('all_on_all_1',config_all,X_train_all_s1,X_test_all_s1,y_train_all_s1,y_test_all_s1)
# ... on native
res_all_on_native_1 = classify_with_input('all_on_native_1',config_all,X_train_native_s1,X_test_native_s1,y_train_native_s1,y_test_native_s1)
# ... on nonnnat
res_all_on_nonnat_1 = classify_with_input('all_on_nonnat_1',config_all,X_train_nonnat_s1,X_test_nonnat_s1,y_train_nonnat_s1,y_test_nonnat_s1)

# Performing the test for configuration 'native' :

# ...on all
res_native_on_all_1    = classify_with_input('native_on_all_1',config_native,X_train_all_s1,X_test_all_s1,y_train_all_s1,y_test_all_s1)
# ... on native
res_native_on_native_1 = classify_with_input('native_on_native_1',config_native,X_train_native_s1,X_test_native_s1,y_train_native_s1,y_test_native_s1)
# ... on nonnnat
res_native_on_nonnat_1 = classify_with_input('native_on_nonnat_1',config_native,X_train_nonnat_s1,X_test_nonnat_s1,y_train_nonnat_s1,y_test_nonnat_s1)

# Performing the test for configuration 'non native' :
# ...on all
res_nonnat_on_all_1    = classify_with_input('nonnat_on_all_1',config_nonnat,X_train_all_s1,X_test_all_s1,y_train_all_s1,y_test_all_s1)
# ... on native
res_nonnat_on_native_1 = classify_with_input('nonnat_on_native_1',config_nonnat,X_train_native_s1,X_test_native_s1,y_train_native_s1,y_test_native_s1)
# ... on nonnnat
res_nonnat_on_nonnat_1 = classify_with_input('nonnat_on_nonnat_1',config_nonnat,X_train_nonnat_s1,X_test_nonnat_s1,y_train_nonnat_s1,y_test_nonnat_s1)


In [74]:
# TRIAL 2

# Performing the test for configuration 'all' :

# ...on all
res_all_on_all_2    = classify_with_input('all_on_all_2',config_all,X_train_all_s2,X_test_all_s2,y_train_all_s2,y_test_all_s2)
# ... on native
res_all_on_native_2 = classify_with_input('all_on_native_2',config_all,X_train_native_s2,X_test_native_s2,y_train_native_s2,y_test_native_s2)
# ... on nonnnat
res_all_on_nonnat_2 = classify_with_input('all_on_nonnat_2',config_all,X_train_nonnat_s2,X_test_nonnat_s2,y_train_nonnat_s2,y_test_nonnat_s2)

# Performing the test for configuration 'native' :

# ...on all
res_native_on_all_2    = classify_with_input('native_on_all_2',config_native,X_train_all_s2,X_test_all_s2,y_train_all_s2,y_test_all_s2)
# ... on native
res_native_on_native_2 = classify_with_input('native_on_native_2',config_native,X_train_native_s2,X_test_native_s2,y_train_native_s2,y_test_native_s2)
# ... on nonnnat
res_native_on_nonnat_2 = classify_with_input('native_on_nonnat_2',config_native,X_train_nonnat_s2,X_test_nonnat_s2,y_train_nonnat_s2,y_test_nonnat_s2)

# Performing the test for configuration 'non native' :
# ...on all
res_nonnat_on_all_2    = classify_with_input('nonnat_on_all_2',config_nonnat,X_train_all_s2,X_test_all_s2,y_train_all_s2,y_test_all_s2)
# ... on native
res_nonnat_on_native_2 = classify_with_input('nonnat_on_native_2',config_nonnat,X_train_native_s2,X_test_native_s2,y_train_native_s2,y_test_native_s2)
# ... on nonnnat
res_nonnat_on_nonnat_2 = classify_with_input('nonnat_on_nonnat_2',config_nonnat,X_train_nonnat_s2,X_test_nonnat_s2,y_train_nonnat_s2,y_test_nonnat_s2)

In [75]:
# TRIAL 3

# Performing the test for configuration 'all' :

# ...on all
res_all_on_all_3    = classify_with_input('all_on_all_3',config_all,X_train_all_s3,X_test_all_s3,y_train_all_s3,y_test_all_s3)
# ... on native
res_all_on_native_3 = classify_with_input('all_on_native_3',config_all,X_train_native_s3,X_test_native_s3,y_train_native_s3,y_test_native_s3)
# ... on nonnnat
res_all_on_nonnat_3 = classify_with_input('all_on_nonnat_3',config_all,X_train_nonnat_s3,X_test_nonnat_s3,y_train_nonnat_s3,y_test_nonnat_s3)

# Performing the test for configuration 'native' :

# ...on all
res_native_on_all_3    = classify_with_input('native_on_all_3',config_native,X_train_all_s3,X_test_all_s3,y_train_all_s3,y_test_all_s3)
# ... on native
res_native_on_native_3 = classify_with_input('native_on_native_3',config_native,X_train_native_s3,X_test_native_s3,y_train_native_s3,y_test_native_s3)
# ... on nonnnat
res_native_on_nonnat_3 = classify_with_input('native_on_nonnat_3',config_native,X_train_nonnat_s3,X_test_nonnat_s3,y_train_nonnat_s3,y_test_nonnat_s3)

# Performing the test for configuration 'non native' :
# ...on all
res_nonnat_on_all_3    = classify_with_input('nonnat_on_all_3',config_nonnat,X_train_all_s3,X_test_all_s3,y_train_all_s3,y_test_all_s3)
# ... on native
res_nonnat_on_native_3 = classify_with_input('nonnat_on_native_3',config_nonnat,X_train_native_s3,X_test_native_s3,y_train_native_s3,y_test_native_s3)
# ... on nonnnat
res_nonnat_on_nonnat_3 = classify_with_input('nonnat_on_nonnat_3',config_nonnat,X_train_nonnat_s3,X_test_nonnat_s3,y_train_nonnat_s3,y_test_nonnat_s3)

In [30]:
# average acc and f1 over the three trials :
# models all_on_....
average_all_on_all = np.array([1./3.*(float(res_all_on_all_1['Model all_on_all_1']['Accuracy']['Overall'])
                                     +float(res_all_on_all_2['Model all_on_all_2']['Accuracy']['Overall'])
                                     +float(res_all_on_all_3['Model all_on_all_3']['Accuracy']['Overall'])),
                               1./3.*(float(res_all_on_all_1['Model all_on_all_1']['F1']['Overall'])
                                     +float(res_all_on_all_2['Model all_on_all_2']['F1']['Overall'])
                                    +float(res_all_on_all_3['Model all_on_all_3']['F1']['Overall']))])
average_all_on_native = np.array([1./3.*(float(res_all_on_native_1['Model all_on_native_1']['Accuracy']['Native'])
                                        +float(res_all_on_native_2['Model all_on_native_2']['Accuracy']['Native'])
                                        +float(res_all_on_native_3['Model all_on_native_3']['Accuracy']['Native'])),
                                  1./3.*(float(res_all_on_native_1['Model all_on_native_1']['F1']['Native'])
                                        +float(res_all_on_native_2['Model all_on_native_2']['F1']['Native']))])
average_all_on_nonnat = np.array([1./3.*(float(res_all_on_nonnat_1['Model all_on_nonnat_1']['Accuracy']['Non_Native'])
                                    +float(res_all_on_nonnat_2['Model all_on_nonnat_2']['Accuracy']['Non_Native'])
                                    +float(res_all_on_nonnat_3['Model all_on_nonnat_3']['Accuracy']['Non_Native'])),
                               1./3.*(float(res_all_on_nonnat_1['Model all_on_nonnat_1']['F1']['Non_Native'])
                                    +float(res_all_on_nonnat_2['Model all_on_nonnat_2']['F1']['Non_Native'])
                                    +float(res_all_on_nonnat_3['Model all_on_nonnat_3']['F1']['Non_Native']))])



In [52]:
# models native_on_....
average_native_on_all = np.array([1./3.*(float(res_native_on_all_1['Model native_on_all_1']['Accuracy']['Overall'])
                                    +float(res_native_on_all_2['Model native_on_all_2']['Accuracy']['Overall'])
                                    +float(res_native_on_all_3['Model native_on_all_3']['Accuracy']['Overall'])),
                               1./3.*(float(res_native_on_all_1['Model native_on_all_1']['F1']['Overall'])
                                    +float(res_native_on_all_2['Model native_on_all_2']['F1']['Overall'])
                                    +float(res_native_on_all_3['Model native_on_all_3']['F1']['Overall']))])
average_native_on_native = np.array([1./3.*(float(res_native_on_native_1['Model native_on_native_1']['Accuracy']['Native'])
                                       +float(res_native_on_native_2['Model native_on_native_2']['Accuracy']['Native'])
                                       +float(res_native_on_native_3['Model native_on_native_3']['Accuracy']['Native'])),
                               1./3.*(float(res_native_on_native_1['Model native_on_native_1']['F1']['Native'])
                                    +float(res_native_on_native_2['Model native_on_native_2']['F1']['Native'])
                                    +float(res_native_on_native_3['Model native_on_native_3']['F1']['Native']))])
average_native_on_nonnat = np.array([1./3.*(float(res_native_on_nonnat_1['Model native_on_nonnat_1']['Accuracy']['Non_Native'])
                                    +float(res_native_on_nonnat_2['Model native_on_nonnat_2']['Accuracy']['Non_Native'])
                                    +float(res_native_on_nonnat_3['Model native_on_nonnat_3']['Accuracy']['Non_Native'])),
                               1./3.*(float(res_native_on_nonnat_1['Model native_on_nonnat_1']['F1']['Non_Native'])
                                    +float(res_native_on_nonnat_2['Model native_on_nonnat_2']['F1']['Non_Native'])
                                    +float(res_native_on_nonnat_3['Model native_on_nonnat_3']['F1']['Non_Native']))])

In [53]:
# models nonnat_on_....

average_nonnat_on_all = np.array([1./3.*(float(res_nonnat_on_all_1['Model nonnat_on_all_1']['Accuracy']['Overall'])
                                    +float(res_nonnat_on_all_2['Model nonnat_on_all_2']['Accuracy']['Overall'])
                                    +float(res_nonnat_on_all_3['Model nonnat_on_all_3']['Accuracy']['Overall'])),
                               1./3.*(float(res_nonnat_on_all_1['Model nonnat_on_all_1']['F1']['Overall'])
                                    +float(res_nonnat_on_all_2['Model nonnat_on_all_2']['F1']['Overall'])
                                    +float(res_nonnat_on_all_3['Model nonnat_on_all_3']['F1']['Overall']))])
average_nonnat_on_native = np.array([1./3.*(float(res_nonnat_on_native_1['Model nonnat_on_native_1']['Accuracy']['Native'])
                                       +float(res_nonnat_on_native_2['Model nonnat_on_native_2']['Accuracy']['Native'])
                                       +float(res_nonnat_on_native_3['Model nonnat_on_native_3']['Accuracy']['Native'])),
                               1./3.*(float(res_nonnat_on_native_1['Model nonnat_on_native_1']['F1']['Native'])
                                    +float(res_nonnat_on_native_2['Model nonnat_on_native_2']['F1']['Native'])
                                    +float(res_nonnat_on_native_3['Model nonnat_on_native_3']['F1']['Native']))])
average_nonnat_on_nonnat = np.array([1./3.*(float(res_nonnat_on_nonnat_1['Model nonnat_on_nonnat_1']['Accuracy']['Non_Native'])
                                    +float(res_nonnat_on_nonnat_2['Model nonnat_on_nonnat_2']['Accuracy']['Non_Native'])
                                    +float(res_nonnat_on_nonnat_3['Model nonnat_on_nonnat_3']['Accuracy']['Non_Native'])),
                               1./3.*(float(res_nonnat_on_nonnat_1['Model nonnat_on_nonnat_1']['F1']['Non_Native'])
                                    +float(res_nonnat_on_nonnat_2['Model nonnat_on_nonnat_2']['F1']['Non_Native'])
                                    +float(res_nonnat_on_nonnat_3['Model nonnat_on_nonnat_3']['F1']['Non_Native']))])

In [54]:
# Create dictionnary of all averages 
dict = {'Parameters -all-': {'Data -all-':    {'Accuracy (avg)':float(average_all_on_all[0]),'F1 (avg)':float(average_all_on_all[1])},
                             'Data -native-': {'Accuracy (avg)':float(average_all_on_native[0]),'F1 (avg)':float(average_all_on_native[1])},
                             'Data -nonnat-': {'Accuracy (avg)':float(average_all_on_nonnat[0]),'F1 (avg)':float(average_all_on_nonnat[1])}},
        'Parameters -native-' : {'Data -all-':    {'Accuracy (avg)':float(average_native_on_all[0]),'F1 (avg)':float(average_native_on_all[1])},
                                 'Data -native-': {'Accuracy (avg)':float(average_native_on_native[0]),'F1 (avg)':float(average_native_on_native[1])},
                                 'Data -nonnat-': {'Accuracy (avg)':float(average_native_on_nonnat[0]),'F1 (avg)':float(average_native_on_nonnat[1])}},
        'Parameters -nonnat-': {'Data -all-':    {'Accuracy (avg)':float(average_nonnat_on_all[0]),'F1 (avg)':float(average_nonnat_on_all[1])},
                                'Data -native-': {'Accuracy (avg)':float(average_nonnat_on_native[0]),'F1 (avg)':float(average_nonnat_on_native[1])},
                                'Data -nonnat-': {'Accuracy (avg)':float(average_nonnat_on_nonnat[0]),'F1 (avg)':float(average_nonnat_on_nonnat[1])}}}


In [55]:
# Save it

with open('Outputs/RESULTS_TEST_STAGE.yaml', 'w') as file:
    yaml.dump(dict, file)

In [76]:
# Create separate dictionnaries for each trial and save them 

dict_1 = {'Parameters -all-': {'Data -all-'   : {'Accuracy':float(res_all_on_all_1['Model all_on_all_1']['Accuracy']['Overall']),'F1':float(res_all_on_all_1['Model all_on_all_1']['F1']['Overall']),
                                                 'Subset Native':{'Accuracy' : float(res_all_on_all_1['Model all_on_all_1']['Accuracy']['Native']),'F1':float(res_all_on_all_1['Model all_on_all_1']['F1']['Native'])},
                                                 'Subset Nonnat':{'Accuracy' : float(res_all_on_all_1['Model all_on_all_1']['Accuracy']['Non_Native']),'F1':float(res_all_on_all_1['Model all_on_all_1']['F1']['Non_Native'])}},
                             'Data -native-': {'Accuracy':float(res_all_on_native_1['Model all_on_native_1']['Accuracy']['Native']),'F1':float(res_all_on_native_1['Model all_on_native_1']['F1']['Native'])},
                             'Data -nonnat-': {'Accuracy':float(res_all_on_nonnat_1['Model all_on_nonnat_1']['Accuracy']['Non_Native']),'F1':float(res_all_on_nonnat_1['Model all_on_nonnat_1']['F1']['Non_Native'])}},
        'Parameters -native-' : {'Data -all-':  {'Accuracy':float(res_native_on_all_1['Model native_on_all_1']['Accuracy']['Overall']),'F1':float(res_native_on_all_1['Model native_on_all_1']['F1']['Overall']),
                                                 'Subset Native':{'Accuracy' : float(res_native_on_all_1['Model native_on_all_1']['Accuracy']['Native']),'F1':float(res_native_on_all_1['Model native_on_all_1']['F1']['Native'])},
                                                 'Subset Nonnat':{'Accuracy' : float(res_native_on_all_1['Model native_on_all_1']['Accuracy']['Non_Native']),'F1':float(res_native_on_all_1['Model native_on_all_1']['F1']['Non_Native'])}},
                                 'Data -native-': {'Accuracy':float(res_native_on_native_1['Model native_on_native_1']['Accuracy']['Native']),'F1':float(res_native_on_native_1['Model native_on_native_1']['F1']['Native'])},
                                 'Data -nonnat-': {'Accuracy':float(res_native_on_nonnat_1['Model native_on_nonnat_1']['Accuracy']['Non_Native']),'F1':float(res_native_on_nonnat_1['Model native_on_nonnat_1']['F1']['Non_Native'])}},
        'Parameters -nonnat-': {'Data -all-':    {'Accuracy':float(res_nonnat_on_all_1['Model nonnat_on_all_1']['Accuracy']['Overall']),'F1':float(res_nonnat_on_all_1['Model nonnat_on_all_1']['F1']['Overall']),
                                                  'Subset Native':{'Accuracy' : float(res_nonnat_on_all_1['Model nonnat_on_all_1']['Accuracy']['Native']),'F1':float(res_nonnat_on_all_1['Model nonnat_on_all_1']['F1']['Native'])},
                                                  'Subset Nonnat':{'Accuracy' : float(res_nonnat_on_all_1['Model nonnat_on_all_1']['Accuracy']['Non_Native']),'F1':float(res_nonnat_on_all_1['Model nonnat_on_all_1']['F1']['Non_Native'])}},
                                'Data -native-': {'Accuracy':float(res_nonnat_on_native_1['Model nonnat_on_native_1']['Accuracy']['Native']),'F1':float(res_nonnat_on_native_1['Model nonnat_on_native_1']['F1']['Native'])},
                                'Data -nonnat-': {'Accuracy':float(res_nonnat_on_nonnat_1['Model nonnat_on_nonnat_1']['Accuracy']['Non_Native']),'F1':float(res_nonnat_on_nonnat_1['Model nonnat_on_nonnat_1']['F1']['Non_Native'])}}}
with open('Outputs/RESULTS_TEST_STAGE_BASELINE_TRIAL_1.yaml', 'w') as file:
    yaml.dump(dict_1, file)

In [77]:
dict_2 = {'Parameters -all-': {'Data -all-'   : {'Accuracy':float(res_all_on_all_2['Model all_on_all_2']['Accuracy']['Overall']),'F1':float(res_all_on_all_2['Model all_on_all_2']['F1']['Overall']),
                                                 'Subset Native':{'Accuracy' : float(res_all_on_all_2['Model all_on_all_2']['Accuracy']['Native']),'F1':float(res_all_on_all_2['Model all_on_all_2']['F1']['Native'])},
                                                 'Subset Nonnat':{'Accuracy' : float(res_all_on_all_2['Model all_on_all_2']['Accuracy']['Non_Native']),'F1':float(res_all_on_all_2['Model all_on_all_2']['F1']['Non_Native'])}},
                             'Data -native-': {'Accuracy':float(res_all_on_native_2['Model all_on_native_2']['Accuracy']['Native']),'F1':float(res_all_on_native_2['Model all_on_native_2']['F1']['Native'])},
                             'Data -nonnat-': {'Accuracy':float(res_all_on_nonnat_2['Model all_on_nonnat_2']['Accuracy']['Non_Native']),'F1':float(res_all_on_nonnat_2['Model all_on_nonnat_2']['F1']['Non_Native'])}},
        'Parameters -native-' : {'Data -all-':    {'Accuracy':float(res_native_on_all_2['Model native_on_all_2']['Accuracy']['Overall']),'F1':float(res_native_on_all_2['Model native_on_all_2']['F1']['Overall']),
                                                 'Subset Native':{'Accuracy' : float(res_native_on_all_2['Model native_on_all_2']['Accuracy']['Native']),'F1':float(res_native_on_all_2['Model native_on_all_2']['F1']['Native'])},
                                                 'Subset Nonnat':{'Accuracy' : float(res_native_on_all_2['Model native_on_all_2']['Accuracy']['Non_Native']),'F1':float(res_native_on_all_2['Model native_on_all_2']['F1']['Non_Native'])}},
                                 'Data -native-': {'Accuracy':float(res_native_on_native_2['Model native_on_native_2']['Accuracy']['Native']),'F1':float(res_native_on_native_2['Model native_on_native_2']['F1']['Native'])},
                                 'Data -nonnat-': {'Accuracy':float(res_native_on_nonnat_2['Model native_on_nonnat_2']['Accuracy']['Non_Native']),'F1':float(res_native_on_nonnat_2['Model native_on_nonnat_2']['F1']['Non_Native'])}},
        'Parameters -nonnat-': {'Data -all-':    {'Accuracy':float(res_nonnat_on_all_2['Model nonnat_on_all_2']['Accuracy']['Overall']),'F1':float(res_nonnat_on_all_2['Model nonnat_on_all_2']['F1']['Overall']),
                                                  'Subset Native':{'Accuracy' : float(res_nonnat_on_all_2['Model nonnat_on_all_2']['Accuracy']['Native']),'F1':float(res_nonnat_on_all_2['Model nonnat_on_all_2']['F1']['Native'])},
                                                  'Subset Nonnat':{'Accuracy' : float(res_nonnat_on_all_2['Model nonnat_on_all_2']['Accuracy']['Non_Native']),'F1':float(res_nonnat_on_all_2['Model nonnat_on_all_2']['F1']['Non_Native'])}},
                                'Data -native-': {'Accuracy':float(res_nonnat_on_native_2['Model nonnat_on_native_2']['Accuracy']['Native']),'F1':float(res_nonnat_on_native_2['Model nonnat_on_native_2']['F1']['Native'])},
                                'Data -nonnat-': {'Accuracy':float(res_nonnat_on_nonnat_2['Model nonnat_on_nonnat_2']['Accuracy']['Non_Native']),'F1':float(res_nonnat_on_nonnat_2['Model nonnat_on_nonnat_2']['F1']['Non_Native'])}}}
with open('Outputs/RESULTS_TEST_STAGE_BASELINE_TRIAL_2.yaml', 'w') as file:
    yaml.dump(dict_2, file)

In [ ]:
dict_3 = {'Parameters -all-': {'Data -all-'   : {'Accuracy':float(res_all_on_all_3['Model all_on_all_3']['Accuracy']['Overall']),'F1':float(res_all_on_all_3['Model all_on_all_3']['F1']['Overall']),
                                                 'Subset Native':{'Accuracy' : float(res_all_on_all_3['Model all_on_all_3']['Accuracy']['Native']),'F1':float(res_all_on_all_3['Model all_on_all_3']['F1']['Native'])},
                                                 'Subset Nonnat':{'Accuracy' : float(res_all_on_all_3['Model all_on_all_3']['Accuracy']['Non_Native']),'F1':float(res_all_on_all_3['Model all_on_all_3']['F1']['Non_Native'])}},
                             'Data -native-': {'Accuracy':float(res_all_on_native_3['Model all_on_native_3']['Accuracy']['Native']),'F1':float(res_all_on_native_3['Model all_on_native_3']['F1']['Native'])},
                             'Data -nonnat-': {'Accuracy':float(res_all_on_nonnat_3['Model all_on_nonnat_3']['Accuracy']['Non_Native']),'F1':float(res_all_on_nonnat_3['Model all_on_nonnat_3']['F1']['Non_Native'])}},
        'Parameters -native-' : {'Data -all-':    {'Accuracy':float(res_native_on_all_3['Model native_on_all_3']['Accuracy']['Overall']),'F1':float(res_native_on_all_3['Model native_on_all_3']['F1']['Overall']),
                                                 'Subset Native':{'Accuracy' : float(res_native_on_all_3['Model native_on_all_3']['Accuracy']['Native']),'F1':float(res_native_on_all_3['Model native_on_all_3']['F1']['Native'])},
                                                 'Subset Nonnat':{'Accuracy' : float(res_native_on_all_3['Model native_on_all_3']['Accuracy']['Non_Native']),'F1':float(res_native_on_all_3['Model native_on_all_3']['F1']['Non_Native'])}},
                                 'Data -native-': {'Accuracy':float(res_native_on_native_3['Model native_on_native_3']['Accuracy']['Native']),'F1':float(res_native_on_native_3['Model native_on_native_3']['F1']['Native'])},
                                 'Data -nonnat-': {'Accuracy':float(res_native_on_nonnat_3['Model native_on_nonnat_3']['Accuracy']['Non_Native']),'F1':float(res_native_on_nonnat_3['Model native_on_nonnat_3']['F1']['Non_Native'])}},
        'Parameters -nonnat-': {'Data -all-':    {'Accuracy':float(res_nonnat_on_all_3['Model nonnat_on_all_3']['Accuracy']['Overall']),'F1':float(res_nonnat_on_all_3['Model nonnat_on_all_3']['F1']['Overall']),
                                                  'Subset Native':{'Accuracy' : float(res_nonnat_on_all_3['Model nonnat_on_all_3']['Accuracy']['Native']),'F1':float(res_nonnat_on_all_3['Model nonnat_on_all_3']['F1']['Native'])},
                                                  'Subset Nonnat':{'Accuracy' : float(res_nonnat_on_all_3['Model nonnat_on_all_3']['Accuracy']['Non_Native']),'F1':float(res_nonnat_on_all_3['Model nonnat_on_all_3']['F1']['Non_Native'])}},
                                'Data -native-': {'Accuracy':float(res_nonnat_on_native_3['Model nonnat_on_native_3']['Accuracy']['Native']),'F1':float(res_nonnat_on_native_3['Model nonnat_on_native_3']['F1']['Native'])},
                                'Data -nonnat-': {'Accuracy':float(res_nonnat_on_nonnat_3['Model nonnat_on_nonnat_3']['Accuracy']['Non_Native']),'F1':float(res_nonnat_on_nonnat_3['Model nonnat_on_nonnat_3']['F1']['Non_Native'])}}}
with open('Outputs/RESULTS_TEST_STAGE_BASELINE_TRIAL_3.yaml', 'w') as file:
    yaml.dump(dict_3, file)